In [1]:
from fastai.vision.all import *
from fastai.callback.fp16 import *

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
path = untar_data(URLs.CIFAR)

In [4]:
image_size = 128
dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                    get_items=get_image_files,
                    get_y=parent_label,
                    item_tfms=Resize(128))

dls = dblock.dataloaders(os.path.join(path.__str__(), "train"), bs=128)

In [5]:
learner = vision_learner(dls=dls,
                        arch=models.resnet18,
                        metrics=accuracy,
                        normalize=True,
                        loss_func=CrossEntropyLossFlat(),
                        pretrained=False).to_fp16()

c:\Users\Jacqu\miniconda3\envs\conda-torch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Jacqu\miniconda3\envs\conda-torch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
learner.lr_find(suggest_funcs=(slide, valley))

In [7]:
learner = vision_learner(dls=dls,
                        lr=0.1,
                        arch=models.resnet18,
                        metrics=accuracy,
                        normalize=True,
                        loss_func=CrossEntropyLossFlat(),
                        pretrained=False).to_fp16()

In [8]:
learner.fine_tune(10, freeze_epochs=3)

epoch,train_loss,valid_loss,accuracy,time
0,2.459501,1.961497,0.314300,02:06
1,1.992125,1.793895,0.351700,01:45
2,1.853238,1.739088,0.366300,01:45


epoch,train_loss,valid_loss,accuracy,time
0,1.514968,1.487120,0.462300,01:45
1,1.303762,1.389856,0.498600,01:44
2,1.094843,1.442565,0.499400,01:45
3,0.754064,1.566370,0.510000,01:45
4,0.408290,1.754561,0.533400,01:45
5,0.202474,2.053349,0.540100,01:45
6,0.091652,2.120739,0.544100,01:45
7,0.045083,2.182244,0.557000,01:45


KeyboardInterrupt: 

In [ ]:
# run on test set
test_files = get_image_files(path / "test")
label = TensorCategory([dls.vocab.o2i[parent_label(f)] for f in test_files])
print(label.shape)

train_files = get_image_files(path / "train")
label_train = TensorCategory([dls.vocab.o2i[parent_label(f)] for f in train_files])
print(label_train.shape)

pred = learner.get_preds(dl=dls.test_dl(test_files))
acc = accuracy(pred[0], label).item()
print(f"test accuracy = {acc}") 